# PDAC Calibration - Double SEER incidence

Author: Sophie Wagner, sw3767@cumc.columbia.edu

In [1]:
# Required Packages
import numpy as np  # For matrix manipulation
import pandas as pd  # For output/input data processing
import matplotlib.pyplot as plt  # For visualizations
from csaps import csaps
from tqdm import tqdm
from datetime import datetime

# Add the src directory to the Python path
import sys
import os
sys.path.append(os.path.abspath('../src'))

# Load .py files
import common_functions as func
import calibration_plots as p
import configs as c
import gof

# Some aesthetic options
np.set_printoptions(suppress=True, linewidth=300, formatter={'float': '{: 0.9f}'.format})
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Set up matrix

In [2]:
def row_normalize(matrix):
    for age_layer in range(matrix.shape[0]):  # Loop over each age layer
        layer = matrix[age_layer]
        for row_idx in range(layer.shape[0]):  # Loop over each row
            row = layer[row_idx]
            
            # Exclude diagonal (stay-in-state) and ACM (all-cause mortality) from scaling
            acm_idx = -1  # Assuming ACM is the last state (adjust index if necessary)
            non_diag_sum = np.sum(row) - row[row_idx] - row[8]  # Exclude diagonal and ACM
            
            if non_diag_sum > 1:  # If the non-ACM probabilities exceed 1
                scaling_factor = 1 / non_diag_sum
                row[:-1] *= scaling_factor  # Scale all except the last column (ACM)
            
            # Adjust diagonal (stay-in-state probability)
            row[row_idx] = max(0, 1 - non_diag_sum - row[8])
    
    return matrix


def create_matrix():
    matrix = np.zeros((len(c.age_layers_1y), len(c.health_states_itos), len(c.health_states_itos)))
    matrix[:15, 0, 1] = 0.000002
    matrix[15:30, 0, 1] = np.linspace(0.00002, 0.0001, 15)
    matrix[30:55, 0, 1] = np.linspace(0.0001, 0.0006, 25)
    matrix[55:, 0, 1] = 0.0006
    matrix[:, 1, 2] = 0.380016  # c.model_inputs_dict['p_Local_to_Regional_PC']
    matrix[:, 2, 3] = 0.666632  # c.model_inputs_dict['p_Regional_to_Distant_PC']
    matrix[:, 1, 4] = 0.040325  # c.model_inputs_dict['p_symptom_local']
    matrix[:, 2, 5] = 0.328861  # c.model_inputs_dict['p_symptom_regional']
    matrix[:, 3, 6] = 0.332143  # c.model_inputs_dict['p_symptom_distant']
    

    matrix = add_acm(matrix)  # ACM
    matrix = add_csd(matrix)  # CSD
    matrix = constrain_matrix(matrix)  # constrain
    matrix = row_normalize(matrix)  # normalize

    return matrix


def constrain_matrix(matrix):
    matrix = np.clip(matrix, 0.000001, 1.0)
    matrix[:, 0, 1] = np.maximum.accumulate(matrix[:, 0, 1])
    matrix[:, 1, 2] = np.maximum(matrix[:, 0, 1], matrix[:, 1, 2])
    matrix[:, 2, 3] = np.maximum(matrix[:, 1, 2], matrix[:, 2, 3])
    matrix[:, 2, 5] = np.maximum(matrix[:, 1, 4], matrix[:, 2, 5])
    matrix[:, 3, 6] = np.maximum(matrix[:, 2, 5], matrix[:, 3, 6])
    
    return matrix


def add_acm(matrix):
    if matrix.shape[0] == 65:
        matrix[:, :7, 8] = c.acm_1y[:65, np.newaxis]  # Healthy to ACM
    else:
        matrix[:, :7, 8] = c.acm_1y[:, np.newaxis]  # Healthy to ACM
    matrix[:, 8, 8] = 1
    return matrix


def add_csd(matrix):
    matrix[:, 4, 7] = int(c.model_inputs_dict['p_local_death'])
    matrix[:, 5, 7] = int(c.model_inputs_dict['p_regional_death'])
    matrix[:, 6, 7] = int(c.model_inputs_dict['p_distant_death'])
    matrix[:, 7, 7] = 1
    return matrix

## Step function

In [3]:
def step(matrix, step_size, num_adj=3):
    step_mat = np.random.choice(len(c.transitions_itos), size=num_adj, replace=True)
    points = list(c.transitions_itos.keys()) 
    step_age = np.random.choice(range(8, 13), size=num_adj, replace=True) # Only select age groups after 60
    small_matrix = matrix[::5, :, :]
    for i in range(num_adj):
        from_state, to_state = points[step_mat[i]]
        step_param = small_matrix[step_age[i], from_state, to_state] * step_size
        small_matrix[step_age[i], from_state, to_state] += np.random.uniform(low=-step_param, high=step_param)
    
    # Update params after age 60
    small_matrix[12, :, :] = np.minimum(small_matrix[11, :, :], small_matrix[12, :, :])
    new_matrix = np.copy(matrix)
    new_matrix[40:,:,:] = csaps([62.5,67.5,72.5,77.5,82.5], small_matrix[8:], axis=0, smooth=0.001)(c.ages_1y[40:])
    assert new_matrix.shape[0]==65, "new_matrix not updated properly, incorrect shape."
    new_matrix = constrain_matrix(new_matrix)
    new_matrix = add_acm(new_matrix)
    new_matrix = add_csd(new_matrix)
    new_matrix = row_normalize(new_matrix)

    return new_matrix

## Run markov

In [11]:
def run_markov(matrix, starting_age=20, max_age=84):
    
    current_age = starting_age
    age_layer = 0
    month_pop, pop_log = c.starting_pop, c.starting_pop  # (13, 1)
    
    inc_log = np.zeros(pop_log.shape)  # to track new incidences in each state
    matrixT = matrix.transpose(0,2,1)  # (65, 13, 13)
    inflow_matrix = np.tril(matrixT, k=-1)
    old_risk, new_risk = 0, 1
    while current_age < max_age:            

        # Matrix multiplication (state transition)
        inflow_mat = inflow_matrix[age_layer] 
        mat = matrixT[age_layer]
        month_inc = np.matmul(inflow_mat, month_pop)  # (9, 9)(9, 1)->(9, 1)
        month_pop = np.matmul(mat, month_pop)  # (9, 9)(9, 1)->(9, 1)
         
        # Add to log
        inc_log = np.concatenate((inc_log, month_inc), axis=1)  # (13, 65)
        pop_log = np.concatenate((pop_log, month_pop), axis=1)  # (13, 65)
        
        current_age += 1
        if age_layer < matrix.shape[0]-1:  # if we run 85-100 stay at 85
            age_layer += 1
            
    incidence, incidence_unadj = inc_log.copy(), inc_log.copy()
    dead_factor = np.divide(c.N, c.N - pop_log[7:, :].sum(axis=0))  # inc and prev denominator is out of living only
    prevalence = np.zeros(pop_log.shape) 

    for state in range(9):
        incidence[state, :] = np.multiply(incidence[state, :], dead_factor)
        prevalence[state, :] = np.multiply(pop_log[state, :], dead_factor)
        
    # assert np.allclose(pop_log[:,:40], c.starting_pop_log[:,:-1], atol=1e-6), "First 40 years mismatch."
    # assert np.allclose(inc_log[:,:40], c.starting_inc_log[:,:-1], atol=1e-6), "First 40 years incidence mismatch."

    return incidence, prevalence, incidence_unadj, pop_log

## Simulated annealing

In [6]:
def simulated_annealing(n_iterations, step_size, start_tmat=None, n_adj=7, verbose=False):

    if start_tmat is None:
        start_tmat = create_matrix()

    best_t = np.copy(start_tmat)
    best_log = run_markov(best_t)
    best_eval = gof.objective(best_log)  # evaluate the initial point
    curr_t, curr_eval = best_t, best_eval  # current working solution
    ticker = 0

    with tqdm(total=n_iterations, desc="Simulated annealing progress", unit="iteration") as pbar:
        
        for i in range(n_iterations):

            # Run model
            candidate_t = np.copy(curr_t)
            candidate_t = step(candidate_t, step_size, n_adj)
            candidate_log = run_markov(candidate_t)
            candidate_eval = gof.objective(candidate_log)  # Evaluate candidate point

            # Update "best" if better than candidate
            if candidate_eval < best_eval:
                ticker = 0
                best_t, best_eval = np.copy(candidate_t), np.copy(candidate_eval)
                best_log = run_markov(best_t)

            else:
                ticker += 1

            # t = 10 / float(i+1)  # calculate temperature for current epoch
            t = 1 / (1 + 0.1 * np.log(i + 1))  # Slower cooling

            # Progress report
            if verbose and i % 1000 == 0:
                log_adj, _, inc_log, _ = best_log
                total_dxd = np.sum(inc_log[4:7, :]) / c.N
                print(i, ": ", best_eval, "   PDAC: ", round(total_dxd, 5),"   tick:",ticker)
                if i % 10000 == 0:
                    transition_probs = p.extract_transition_probs(best_t)
                    print(f"Progress report, i = {i}")
                    print(transition_probs)
                    p.plot_vs_seer(log_adj, c.seer_inc)

            # Check if we should update "curr"
            diff = (candidate_eval - curr_eval)  # difference between candidate and current point evaluation
            metropolis = np.exp(-diff / t)  # calculate metropolis acceptance criterion
            if (diff < 0 or np.random.random() < metropolis):  # check if we should keep the new point
                curr_t, curr_eval = np.copy(candidate_t), np.copy(candidate_eval)  # store the new current point
                ticker = 0

            pbar.update(1)

    print(best_eval)
    
    return best_t

## Run 

In [13]:
def run_sa(tmat=None, save_all=False):
    if tmat is None:
        tmat = create_matrix()
    result = simulated_annealing(n_iterations=100000, step_size=0.25, start_tmat=tmat, n_adj=5, verbose=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    
    curr_tmat = result.copy()
    curr_log = run_markov(curr_tmat)
    log_adj, log_prev, log_inc, pop_log = curr_log

    # Extract transition probabilities
    transition_probs = p.extract_transition_probs(curr_tmat)

    # Saving
    if save_all:
        # Save the with the timestamp in the filenames
        output_dir = c.OUTPUT_PATHS[type]
        log_path, tmat_path, plot_path = c.OUTPUT_PATHS["logs"], c.OUTPUT_PATHS["tmats"], c.OUTPUT_PATHS["plots"]
        np.save(f"{tmat_path}/{timestamp}_tmat.npy", curr_tmat)
        pd.DataFrame(log_adj).to_csv(f"{log_path}/{timestamp}_inc_adj.csv")
        pd.DataFrame(log_prev).to_csv(f"{log_path}/{timestamp}_prev.csv")
        pd.DataFrame(log_inc).to_csv(f"{log_path}/{timestamp}_inc_unadj.csv")
        transition_probs.to_csv(f"{log_path}/{timestamp}_tps.csv")
        p.plot_vs_seer(curr_log, c.seer_inc, save_imgs=True, outpath=plot_path, timestamp=timestamp)
        p.plot_vs_seer_total(curr_log, c.seer_inc, save_imgs=True, outpath=plot_path, timestamp=timestamp)

    else:
        print(transition_probs)
        p.plot_vs_seer(log_adj, c.seer_inc)
        p.plot_vs_seer_total(log_adj, c.seer_inc)
        print(pd.DataFrame(pop_log.T))

    return curr_tmat

In [8]:
tmat = np.load("../out/tmats/20241205_1310_tmat.npy")

In [ ]:
log = run_markov(tmat)
p.plot_vs_seer(log[0], c.seer_inc)